In [1]:
__file__=''
import os,sys
import pandas as pd
sys.path.append(os.path.join(os.path.dirname(__file__),'../LIB/'))
sys.path.append(os.path.join(os.path.dirname(__file__),'../../../../automl/automl_libs/'))
from env import FILE
import utils
from itertools import combinations
from feature_engineering import *
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder

In [2]:
# X = pd.read_pickle(FILE.shiyi_fillna_ori.value)


In [3]:
train = pd.read_pickle(FILE.train_ori.value)
print('train shape is: {}'.format(train.shape))
test = pd.read_pickle(FILE.test_ori.value)
print('test shape is: {}'.format(test.shape))

X = pd.concat([train.drop(['click'],axis=1),test])
print(X.shape)

X = X.sort_values('time')
X = X.reset_index(drop=True)

X_shiyi = pd.read_pickle(FILE.shiyi_fillna_ori.value)
print(X_shiyi.shape)

X = X.merge(X_shiyi[['time_hour','instance_id']],how='inner',on='instance_id')

X = X.sort_values('time')
X = X.reset_index(drop=True)

print(X.shape)

train shape is: (1001650, 35)
test shape is: (40024, 34)
(1041674, 34)
(1041674, 45)
(1041674, 35)


# Step 1, change data type

In [4]:
for col in X.columns:
    if X[col].dtype == 'float64':
        X[col] = X[col].astype(utils.set_type(X[col],'float'))
    if X[col].dtype == 'int64':
        X[col] = X[col].astype(utils.set_type(X[col],'int'))
        
print(sys.getsizeof(X)/1024**3)

0.785621888935566


# Step 2, sort user tags, to make sure it is ordered consistently

In [5]:
def tags_sort(x):
    try:
        return ','.join(sorted(list(set(x.split(',')) -set(['']))))
    except:
        return x
X['user_tags'] = X['user_tags'].apply(tags_sort)



# Step 3, creative_size, encoded by 'creative_width' and 'creative_height'

In [6]:
X['creative_size'] = X['creative_width'].astype(str) + '*' + X['creative_height'].astype(str)
X.drop(['creative_width', 'creative_height'],axis=1,inplace=True)

# Step 4, drop those only has 1 unique value

In [7]:
drop_col = []
for col in X.columns:
    if X[col].nunique() == 1:
        drop_col.append(col)
        
print(drop_col)

X.drop(drop_col,axis=1,inplace=True)

['creative_is_js', 'creative_is_voicead', 'app_paid']


# Define categorical columns

In [8]:
not_categorical = ['instance_id','time', 'datetime']
categorical_cols = list(set(X.columns) - set(not_categorical))
print(len(categorical_cols))


29


# Define Function List

In [9]:
count_func_list = [count,cumulative_count,reverse_cumulative_count]
time_next_func_list = [time_to_n_next,time_to_n_previous]
count_time_func_list = [count_in_previous_n_time_unit,count_in_next_n_time_unit]

# FillNa

In [10]:
le = LabelEncoder()
#X['user_tags'] =  X['user_tags'].fillna('naNotFound')
# X['user_tags'] = le.fit_transform(X['user_tags'])

for col in tqdm(categorical_cols):
    if X[col].dtype == 'object':
        X[col] = X[col].fillna('FillNAValue')
        pass
    else:
        X[col] = X[col].fillna(-100000)
    X[col] = le.fit_transform(X[col])

100%|██████████| 29/29 [00:13<00:00,  2.09it/s]


# Aggregation

In [11]:
def get_new_name(cob_col,func):
    return '+'.join(cob_col)+'_'+func.__name__

In [12]:
def auto_generating(X,categorical_cols,
                    count_func_list,time_next_func_list,count_time_func_list,
                    dummy_col='instance_id',time_col='time',n_comb=1,
                    saving_path_count=None,
                    saving_path_timeNext=None,
                    saving_path_timeCount=None):
    
    if saving_path_count is not None:
        try:
            result_count = pd.read_pickle(saving_path_count)
        except:
            result_count = pd.DataFrame()
            
    if saving_path_timeNext is not None:
        try:
            result_time_next = pd.read_pickle(saving_path_timeNext)
        except:
            result_time_next = pd.DataFrame()
            
    if saving_path_timeCount is not None:
        try:
            result_time_count = pd.read_pickle(saving_path_timeCount)
        except:
            result_time_count = pd.DataFrame()

    counter = 0
    for col in combinations(categorical_cols,n_comb):
        col = sorted(col)
        print('#######   {}/{}   ########'.format(col,counter))
        col = list(col)
        counter+=1
        need_save_result_count = False
        need_save_result_time_next = False
        need_save_result_time_count = False
        
        for func in count_func_list:
            feature_name = get_new_name(col,func)
            if feature_name in result_count.columns:
                continue
            result_count[feature_name] = func(X,col,dummy_col,feature_name)[feature_name]
            need_save_result_count = True

        for func in time_next_func_list:

            feature_name = get_new_name(col,func)
            col_time = col.copy()
            col_time.append(time_col)
            if feature_name in result_time_next.columns:
                continue
            result_time_next[feature_name] = func(X,col_time,dummy_col,feature_name)[time_col]
            need_save_result_time_next = True


        for func in count_time_func_list:

            feature_name = get_new_name(col,func)
            col_time = col.copy()
            col_time.append(time_col)
            if feature_name+'_1hour' not in result_time_count.columns:
                result_time_count[feature_name+'_1hour'] =func(X,col_time,dummy_col=dummy_col,
                                                         generated_feature_name=feature_name,
                                                         params={'n':60*60*1})[feature_name]
                need_save_result_time_count = True
            if feature_name+'_6hour' not in result_time_count.columns:
                result_time_count[feature_name+'_6hour'] =func(X,col_time,dummy_col=dummy_col,
                                                         generated_feature_name=feature_name,
                                                         params={'n':60*60*6})[feature_name]
                need_save_result_time_count = True

            if feature_name+'_24hour' not in result_time_count.columns:
                result_time_count[feature_name+'_24hour'] =func(X,col_time,dummy_col=dummy_col,
                                                         generated_feature_name=feature_name,
                                                         params={'n':60*60*6})[feature_name]
                need_save_result_time_count = True
                
        if saving_path_count is not None and need_save_result_count:
            result_count[dummy_col]=X[dummy_col].copy()
            print('saving counts')
            result_count.to_pickle(saving_path_count)
        else:
            print('skip counts')

        if saving_path_timeNext is not None and need_save_result_time_next:
            result_time_next[dummy_col]=X[dummy_col].copy()
            print('saving time next')
            result_time_next.to_pickle(saving_path_timeNext)
        else:
            print('skip time next')

        if saving_path_timeCount is not None and need_save_result_time_count:
            result_time_count[dummy_col]=X[dummy_col].copy()
            print('saving time count in next unit')
            result_time_count.to_pickle(saving_path_timeCount)
        
        else:
            print('skip time count in next unit')
            
    

In [13]:
dummy_col = 'instance_id'
time_col = 'time'
X = X.sort_values(time_col)

for n_combs in [2]:
    saving_path_count = FILE.X_fe_agg_count_formater.value.format(n_combs)
    saving_path_timeNext = FILE.X_fe_agg_time_next_formater.value.format(n_combs)
    saving_path_timeCount = FILE.X_fe_agg_time_count_formater.value.format(n_combs)
    print(saving_path_count)
    print(saving_path_timeNext)
    print(saving_path_timeCount)
    
    auto_generating(X,
                    categorical_cols,
                    count_func_list,
                    time_next_func_list,
                    count_time_func_list,
                    dummy_col,
                    time_col,
                    n_comb=n_combs,
                    saving_path_count=saving_path_count,
                    saving_path_timeNext=saving_path_timeNext,
                    saving_path_timeCount=saving_path_timeCount)

../LIB/../../data/features/X_fe_agg_3_count.pkl
../LIB/../../data/features/X_fe_agg_3_time_next.pkl
../LIB/../../data/features/X_fe_agg_3_time_count.pkl
#######   ('os_name', 'app_cate_id', 'creative_tp_dnf')/0   ########
saving counts
saving time next
saving time count in next unit


KeyboardInterrupt: 